# Projeto guiado: Analisando dados do CIA Factbook usando SQL (Dataquest)

Treino de análise realizada com SQL.

Neste projeto, foi trabalhado os dados do CIA World Factbook, um compêndio de estatísticas sobre todos os países da Terra. O Factbook contém informações demográficas como:

* population- A população a partir de 2015.
* population_growth - A taxa de crescimento anual da população, como porcentagem.
* area - A área total de terra e água.

O arquivo para análise pode ser baixado em: https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db
As consultas foram realizadas de 2 formas distintas, utilizando o ipython-sql e também o pandas

# Descrições para algumas das colunas do dataset:
* name - o nome do país.
* area - A área total terrestre e marítima do país.
* population - população do país.
* population_growth- Crescimento populacional do país como porcentagem.
* birth_rate - A taxa de natalidade do país ou o número de nascimentos por ano por 1.000 pessoas.
* death_rate - A taxa de mortalidade do país ou o número de mortes por ano por 1.000 pessoas.
* area- A área total do país (terra e água).
* area_land- A área do país em quilômetros quadrados .
* area_water - Área aquática do país em quilômetros quadrados.

In [15]:
# Instalando o ipython-sql no notebook
!pip install ipython-sql

  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13704 sha256=0f022110ccd1b7da00049ce2f8e7c47913f9b32c515b2cdf04a1fb5401ab8b17
  Stored in directory: c:\users\marcella\appdata\local\pip\cache\wheels\8c\76\0b\eb9eb3da7e2335e3577e3f96a0ae9f74f206e26457bd1a2bc8
Successfully built prettytable
You should consider upgrading via the 'c:\users\marcella\appdata\local\continuum\anaconda3\python.exe -m pip install --upgrade pip' command.


In [18]:
# Carregando as bibliotecas utilizadas
import pandas as pd
import sqlite3
%%capture
%load_ext sql # carregando o ipython-sql

UsageError: Line magic function `%%capture` not found.


In [10]:
# Criando a conexão do BD 
con = sqlite3.connect("factbook.db")

# Lendo os resultados da consulta sqlite em um DataFrame do pandas
# Retornando os 5 primeiros dados da tabela "facts"
df = pd.read_sql_query("SELECT * FROM facts limit 5;", con)
df.head()

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
1,2,al,Albania,28748,27398,1350,3029278,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
3,4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [27]:
# conectando ao banco com o ipython-sql
%sql sqlite:///factbook.db 

# Retornando os 5 primeiros dados da tabela "facts"
%sql SELECT * FROM facts limit 5;

* sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [13]:
# Calculando em uma única consulta a população mínima, população máxima, crescimento mínimo da população e crescimento máximo da população (usando pandas)
df = pd.read_sql_query("SELECT MIN(population) min_pop, MAX(population) max_pop, MIN(population_growth) min_pop_grwth, MAX(population_growth) max_pop_grwth                    FROM facts;",con)
df

,min_pop,max_pop,min_pop_grwth,max_pop_grwth
0,0,7256490011,0.0,4.02


In [39]:
# Calculando em uma única consulta a população mínima, população máxima, crescimento mínimo da população e crescimento máximo da população (usando ipython-sql)
%sql SELECT MIN(population) min_pop, MAX(population) max_pop, MIN(population_growth) min_pop_grwth, MAX(population_growth) max_pop_grwth FROM facts;

* sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_grwth,max_pop_grwth
0,7256490011,0.0,4.02


In [40]:
# o (s) país (es) com a população máxima
df = pd.read_sql_query("SELECT * FROM facts WHERE population == (SELECT MAX(population) FROM facts);",con)
df

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


In [37]:
# o (s) país (es) com a população máxima
%%sql
SELECT * FROM facts
WHERE population == (
    SELECT MAX(population) FROM facts
);

* sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


In [41]:
# o (s) país (es) com a população mínima
df = pd.read_sql_query("SELECT * FROM facts WHERE population == (SELECT MIN(population) FROM facts);",con)
df

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [36]:
#  O (s) país (es) com a população mínima
%%sql
SELECT * FROM facts
WHERE population == (
    SELECT MIN(population) FROM facts
);

* sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [48]:
# valor médio para as seguintes colunas: "population" e "area"
df = pd.read_sql_query("SELECT AVG(population) avg_population, AVG(area) avg_area FROM facts WHERE name <> 'World';",con)
df

,avg_population,avg_area
0,3.224267e+07,555093.546185


In [47]:
# valor médio para as seguintes colunas: "population" e "area"
%sql SELECT AVG(population) avg_population, AVG(area) avg_area FROM facts WHERE name <> 'World';

* sqlite:///factbook.db
Done.


avg_population,avg_area
32242666.56846473,555093.546184739


In [51]:
# Países em que "population" está acima da média e "area" está abaixo da média.
df = pd.read_sql_query("SELECT * FROM facts WHERE population > (SELECT AVG(population) FROM facts) AND area < (SELECT AVG(area) FROM facts);",con)
df

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,14,bg,Bangladesh,148460,130170,18290,168957745,1.60,21.14,5.61,0.46
1,65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
2,85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.00
3,138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
4,173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.80,0.00
5,185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
6,192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.30


In [50]:
# Países em que "population" está acima da média e "area" está abaixo da média.
%sql SELECT * FROM facts WHERE population > (SELECT AVG(population) FROM facts) AND area < (SELECT AVG(area) FROM facts)

* sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3
